# 📓 The GenAI Revolution Cookbook

**Title:** How to Build a Stateful AI Agent with LangGraph Step-by-Step

**Description:** Build reliable, stateful AI agents with LangGraph using step-by-step patterns, visual debugging, and persistence—ready for tool use and production today.

**📖 Read the full article:** [How to Build a Stateful AI Agent with LangGraph Step-by-Step](https://blog.thegenairevolution.com/article/how-to-build-a-stateful-ai-agent-with-langgraph-step-by-step-3)

---

*This jupyter notebook contains executable code examples. Run the cells below to try out the code yourself!*



When you're building AI agents that need to make decisions, call tools, and keep track of context across multiple turns, orchestration becomes absolutely critical. LangGraph gives you a structured way to define agent workflows as state machines, which means you get fine\-grained control over how your agent reasons, acts, and responds.

In this tutorial, we're going to build a travel assistant agent that helps users plan trips by calling external tools to fetch weather data and search for flights. You'll learn how to define a stateful graph, integrate tool\-calling logic, and trace execution step\-by\-step. By the end, you'll have a working agent that can handle multi\-turn conversations and coordinate multiple tools to deliver genuinely useful results.

## Why This Approach Works

LangGraph treats agent workflows as explicit graphs where each node represents a step (like calling the LLM or executing a tool) and edges define transitions. This makes complex agent behavior so much easier to reason about, debug, and extend compared to implicit loops or callback\-based systems.

Here's what I've found particularly valuable:

* **Explicit state management** – You define exactly what data flows between steps. This makes tracking context and debugging issues straightforward instead of hunting through mysterious state mutations.
* **Composable logic** – Each node is just a function. You can test individual components, swap them out, or extend them without having to rewrite your entire agent from scratch.
* **Built\-in tracing** – LangGraph logs every single state transition, so you can inspect what the agent did at each step and, more importantly, understand why it made those decisions.

This approach really shines when your agent needs to call multiple tools, handle conditional logic, or maintain conversation history across turns. I've found it particularly useful when building agents that need to coordinate between different data sources.

## High\-Level Overview

Let me walk you through how the system actually works:

1. **User input** – The user sends a message like "Find me flights to Tokyo and check the weather"
2. **LLM reasoning** – The agent calls the LLM, which decides whether to respond directly or invoke tools
3. **Tool execution** – If tools are needed, the agent executes them (search\_flights, get\_weather) and collects results
4. **LLM synthesis** – The agent sends tool results back to the LLM, which generates a final response
5. **Output** – The user receives a natural language answer informed by real data

The graph has three main nodes that do the heavy lifting:

* **Agent node** – Calls the LLM to decide next actions
* **Tool node** – Executes requested tools and returns results
* **Conditional edge** – Routes to tools if needed, or ends if the agent is done

## Setup \& Installation

This code runs in Google Colab or any Python 3\.10\+ environment. Let's install the dependencies:

In [ ]:
!pip install -qU langgraph langchain-openai langchain

Set your OpenAI API key:

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "your-openai-api-key"

Replace "your\-openai\-api\-key" with your actual key. And honestly, for production, please use environment variables or secret management tools instead of hardcoding keys. I learned this the hard way in a previous project.

## Step 1: Define Tools

Tools are Python functions decorated with @tool. The LLM can call these functions when it needs external data.

This example defines two tools: one for searching flights and one for fetching weather. Both return mock data for demonstration purposes.

In [ ]:
from langchain_core.tools import tool

@tool
def search_flights(origin: str, destination: str, date: str) -> str:
    """Search for available flights between two cities on a given date."""
    return f"Found 3 flights from {origin} to {destination} on {date}: Flight A ($450), Flight B ($520), Flight C ($610)."

@tool
def get_weather(city: str) -> str:
    """Get current weather information for a city."""
    return f"Weather in {city}: 22°C, partly cloudy, light breeze."

tools = [search_flights, get_weather]

In a real application, you'd replace the return statements with actual API calls to services like Amadeus for flights or OpenWeatherMap for weather data.

## Step 2: Bind Tools to the LLM

The LLM needs to know which tools are available and how to call them. Use .bind\_tools() to attach tool schemas to the model.

In [ ]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o", temperature=0)
model_with_tools = model.bind_tools(tools)

Now when you call model\_with\_tools, the LLM can decide to invoke search\_flights or get\_weather based on what the user's actually asking for.

## Step 3: Define the Agent State

State is a dictionary that flows through the graph. It holds the conversation history and any other data you need to track.

In [ ]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages

class State(TypedDict):
    messages: Annotated[list, add_messages]

The add\_messages annotation tells LangGraph to append new messages to the list rather than replacing it. This is crucial – it preserves conversation history across turns so your agent doesn't forget what was discussed earlier.

## Step 4: Build the Agent Node

The agent node calls the LLM with the current message history. The LLM returns either a text response or a request to call tools.

In [ ]:
def call_agent(state: State):
    """Invoke the LLM with the current conversation state."""
    response = model_with_tools.invoke(state["messages"])
    return {"messages": [response]}

This function takes the state, passes state\["messages"] to the model, and returns the model's response wrapped in a dictionary. LangGraph then merges it back into the state automatically.

## Step 5: Build the Tool Node

LangGraph provides a ToolNode that automatically executes any tools the LLM requested and formats the results as messages.

In [ ]:
from langgraph.prebuilt import ToolNode

tool_node = ToolNode(tools)

When the agent node returns a message with tool\_calls, the graph routes to this node. It runs the tools and appends their outputs to the message list. Pretty straightforward.

## Step 6: Define Routing Logic

After the agent node runs, the graph needs to decide: should it call tools, or is the agent done?

This function checks if the last message contains tool calls. If yes, route to the tool node. If no, end the conversation.

In [ ]:
from langgraph.graph import END

def should_continue(state: State):
    """Determine whether to call tools or finish."""
    last_message = state["messages"][-1]
    if last_message.tool_calls:
        return "tools"
    return END

## Step 7: Assemble the Graph

Now we combine the nodes and edges into a state graph.

In [ ]:
from langgraph.graph import StateGraph, START

workflow = StateGraph(State)

workflow.add_node("agent", call_agent)
workflow.add_node("tools", tool_node)

workflow.add_edge(START, "agent")
workflow.add_conditional_edges("agent", should_continue, {"tools": "tools", END: END})
workflow.add_edge("tools", "agent")

graph = workflow.compile()

Let me break down what each line does:

* `add_node("agent", call_agent)` – Registers the agent node
* `add_node("tools", tool_node)` – Registers the tool execution node
* `add_edge(START, "agent")` – The graph always starts at the agent node
* `add_conditional_edges("agent", should_continue, ...)` – After the agent runs, route to tools or end based on should\_continue
* `add_edge("tools", "agent")` – After tools run, return to the agent so it can synthesize results

## Step 8: Run the Agent

Invoke the graph with a user message. The agent will decide which tools to call and return a final answer.

In [ ]:
from langchain_core.messages import HumanMessage

user_input = "Find me flights from San Francisco to Tokyo on March 15th and tell me the weather in Tokyo."
result = graph.invoke({"messages": [HumanMessage(content=user_input)]})

The result dictionary contains the full message history – the user's input, tool calls, tool results, and the agent's final response. Everything's there for you to inspect.

## Step 9: Display Results and Trace

To see the final answer and understand what happened at each step, print the last AI message and trace all messages.

This helper function extracts the final AI response and prints a numbered trace showing each message type, content, and metadata like tool calls or tool names.

In [ ]:
from langchain_core.messages import AIMessage, ToolMessage

def print_message_trace(result):
    final = [m for m in result["messages"] if isinstance(m, AIMessage)][-1]
    print(final.content)

    print("\nFull Trace:")
    for i, m in enumerate(result["messages"], 1):
        role = type(m).__name__
        meta = ""
        if isinstance(m, AIMessage) and getattr(m, "tool_calls", None):
            meta = f" tool_calls={m.tool_calls}"
        if isinstance(m, ToolMessage):
            meta = f" tool_name={m.name}"
        print(f"{i:02d}. {role}: {m.content}{meta}")

print_message_trace(result)

This trace shows the agent called both tools in parallel, received results, and synthesized a final answer. Actually, the parallel execution is one of my favorite features – it really speeds things up when you need multiple data sources.

## Run and Validate

Test the agent with different inputs to confirm it routes correctly.

Single tool call:

In [ ]:
result = graph.invoke({"messages": [HumanMessage(content="What's the weather in Paris?")]})
print_message_trace(result)

No tool call (direct answer):

In [ ]:
result = graph.invoke({"messages": [HumanMessage(content="What is LangGraph?")]})
print_message_trace(result)

Multi\-turn conversation:

In [ ]:
result = graph.invoke({"messages": [HumanMessage(content="Find flights to Berlin on April 10th.")]})
result = graph.invoke({"messages": result["messages"] + [HumanMessage(content="What about the weather there?")]})
print_message_trace(result)

In the multi\-turn example, the agent maintains context from the first turn and knows "there" refers to Berlin. This is exactly the kind of contextual awareness that makes agents feel more natural to interact with.

## Conclusion

You've built a stateful LangGraph agent that orchestrates tool calls and maintains conversation context. Here's what I think are the key takeaways:

* **State graphs make agent logic explicit** – You control exactly when the LLM is called, when tools run, and how results flow back. No more mysterious behavior buried in callbacks.
* **Tool binding is straightforward** – Decorate functions with @tool and bind them to the model. The LLM handles the rest. It's honestly simpler than I expected when I first started working with this.
* **Tracing is built\-in** – Every message and tool call is logged, making debugging and optimization so much easier than traditional approaches.

For readers working on data extraction challenges, our guide on building a structured data extraction pipeline with LLMs offers complementary strategies for handling unstructured inputs.

If you encounter unexpected model behavior, subtle bugs often stem from tokenization issues—see our article on tokenization pitfalls and invisible characters for actionable solutions.

When scaling to long\-context applications, be aware of memory limitations. Our analysis of context rot and memory management in LLMs explains why models sometimes lose track of earlier information and how to mitigate it.

## Next Steps

* **Add real APIs** – Replace mock data with live calls to flight search APIs (like Amadeus) and weather services (OpenWeatherMap works great)
* **Persist state** – Use LangGraph's checkpointing to save conversation history to a database, enabling multi\-session continuity
* **Add error handling** – Wrap tool calls in try\-except blocks and return user\-friendly error messages when APIs fail. Trust me, they will fail at some point.
* **Deploy as an API** – Serve the graph via FastAPI or Flask so users can interact with it through a web interface or chat app